<a href="https://colab.research.google.com/github/Juhwan01/DeepDive/blob/main/KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B8%B0%EA%B3%84_%EB%8F%85%ED%95%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 이전과 똑같이 레거시 모드로
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [4]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2025-07-23 13:13:07--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  59.5MB/s    in 0.6s    

2025-07-23 13:13:08 (59.5 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2025-07-23 13:13:09--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[=====



# `open(path, 'rb') as f` 설명

## 📌 `open()` 함수 기본 구조

```python
open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
```



## 1. `path` - 파일 경로 지정

```python
open('data.json', 'rb')             # 상대 경로
open('/home/user/data.json', 'rb')  # 절대 경로 (Linux/Mac)
open(r'C:\Users\data.json', 'rb')   # 절대 경로 (Windows)
open(Path('data.json'), 'rb')       # Path 객체 사용
```



## 2. `'rb'` - 모드 설정 (읽기 + 바이너리)

### 📜 파일 모드 정리

| 모드     | 설명          | 사용 예시           |
| ------ | ----------- | --------------- |
| `'r'`  | 텍스트 읽기 (기본) | `.txt`, `.json` |
| `'w'`  | 텍스트 쓰기      | 로그 덮어쓰기 등       |
| `'a'`  | 텍스트 추가      | 로그 추가           |
| `'rb'` | **바이너리 읽기** | 이미지, JSON 등     |
| `'wb'` | 바이너리 쓰기     | 이진 파일 생성        |
| `'ab'` | 바이너리 추가     | 로그 추가 (이진)      |


## 3. `as f` - 파일 객체 변수 할당

```python
with open(path, 'rb') as f:           # 일반적인 사용
with open(path, 'rb') as file:        # file로 이름 지정
with open(path, 'rb') as data_file:   # 의미 부여
```


## 4. 📦 'rb' 모드 자세히 알아보기

### ✅ 바이너리 모드란?

* **바이트 단위**로 읽기
* **인코딩 처리 없이 원본 그대로**
* 모든 파일 형식 가능 (텍스트, 이미지, 오디오 등)

### 🆚 텍스트 모드 vs 바이너리 모드

```python
# 텍스트 모드
with open('text.txt', 'r') as f:
    content = f.read()
    print(type(content))  # <class 'str'>

# 바이너리 모드
with open('text.txt', 'rb') as f:
    content = f.read()
    print(type(content))  # <class 'bytes'>
```



## 5. JSON 파일에서 `'rb'` 쓰는 이유

### ❌ 문제 상황

```python
# 기본 인코딩은 플랫폼 따라 달라 오류 발생 가능
with open('korean.json', 'r') as f:
    data = json.load(f)  # UnicodeDecodeError 발생 가능
```

### ✅ 안전한 방법들

#### 방법 1: 바이너리 모드 (`json.load()`가 자동 처리)

```python
with open('korean.json', 'rb') as f:
    data = json.load(f)
```

#### 방법 2: 텍스트 모드 + 명시적 인코딩

```python
with open('korean.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
```


## 6. 🧪 실제 예제

```python
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
        return squad_dict

# 사용 예시
data = read_squad('squad_data.json')
print(type(data))  # <class 'dict'>
```



## 7. 파일 객체 `f`의 주요 메서드

| 메서드            | 설명              |
| -------------- | --------------- |
| `f.read()`     | 전체 내용 읽기        |
| `f.read(100)`  | 100바이트만 읽기      |
| `f.readline()` | 한 줄씩 읽기         |
| `f.seek(0)`    | 파일 포인터를 처음으로 이동 |
| `f.tell()`     | 현재 포인터 위치 반환    |


## ✅ 결론: JSON 읽을 때 추천 방식

```python
# 방법 1: 바이너리 모드
with open(path, 'rb') as f:
    data = json.load(f)

# 방법 2: 텍스트 모드 + UTF-8 인코딩
with open(path, 'r', encoding='utf-8') as f:
    data = json.load(f)
```



In [5]:
def read_squad(path):
  # 입력 받은 경로를 Path 객체로 변환
  path = Path(path)
  # with 문은 Context Manager를 사용해서 리소스를 안전하게 관리하는 구문
  with open(path,'rb') as f:
    squad_dict = json.load(f)
    # 여기서 자동으로 f.close()가 호출됨!

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
    for passage in group['paragraphs']:
      # context = 본문
      context = passage['context']
      # qa = 질문셋
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [6]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(val_contexts))
print('테스트 데이터의 질문 개수 :', len(val_questions))
print('테스트 데이터의 답변 개수 :', len(val_answers))

훈련 데이터의 본문 개수 : 60407
훈련 데이터의 질문 개수 : 60407
훈련 데이터의 답변 개수 : 60407
테스트 데이터의 본문 개수 : 5774
테스트 데이터의 질문 개수 : 5774
테스트 데이터의 답변 개수 : 5774


In [7]:
print('첫 번째 샘플의 본문')
print('-'*20)
print(train_contexts[0])

첫 번째 샘플의 본문
--------------------
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [8]:
print('첫 번째 샘플의 질문')
print('-'*20)
print(train_questions[0])

첫 번째 샘플의 질문
--------------------
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [9]:
# answer_start는 모델이 예측한 정답이 본문 어디서 시작하는지를 알려주는 위치 정보
print('첫 번째 샘플의 답변')
print('-'*20)
print(train_answers[0])
# answer_start만 있고 answer_end는 없지만, 일반적으로 이렇게 계산
# answer_end = answer_start + len(answer_text)

첫 번째 샘플의 답변
--------------------
{'text': '교향곡', 'answer_start': 54}


In [10]:
# 종료 인덱스 추가
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    # answer뒤에 공백이 있으면 제거 -> 위에서 말한 식처럼 연산해야해서 빈칸까지 카운트 되면 문제가 생기기 때문
    answer['text'] = answer['text'].rstrip()

    # answer_end = answer_start + len(answer_text)
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러 발생"

    answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [11]:
# 다시 조회에서 수정 확인
print('첫 번째 샘플의 답변')
print('-'*20)
print(train_answers[0])

첫 번째 샘플의 답변
--------------------
{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


In [12]:
# 실제로 인덱스가 맞는지 확인
train_contexts[0][54:57]

'교향곡'

In [13]:
# 본문과 질문 두 가지를 토크나이저의 입력으로 사용
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')
# 예전에 Hugging Face transformers 라이브러리를 쓸 땐 보통 이렇게 직접 encode_plus()를 썼었다 -> 전에 실습 자료에서 인코딩 과정에서 encode_plus를 썻던 것이 기억나서 작성
# 문장쌍 인코딩 과정
# 여기서 토크나이저가 model_max_length = 512 이 뒤로 잘리니 정답 라벨이 512 뒤에 있으면 문제 발생
# [CLS] 본문 [SEP] 질문 [SEP]
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(




### ✅ `BatchEncoding` 구조

```markdown
BatchEncoding
├── input_ids             → List[List[int]]
├── attention_mask        → List[List[int]]
├── token_type_ids        → List[List[int]] (문장쌍 있을 때만)
├── offset_mapping        → List[List[Tuple[int, int]]] (옵션)
├── special_tokens_mask   → List[List[int]] (옵션)
├── overflow_to_sample_mapping → List[int] (옵션)
├── ...                   → 기타 항목들
└── [i]                   → EncodingFast 객체로 접근 가능
```



### ✅ `EncodingFast` 구조 (`BatchEncoding[i]`)

```markdown
EncodingFast
├── .tokens             → List[str]             # 토크나이저가 생성한 토큰들
├── .ids                → List[int]             # 각 토큰의 ID
├── .offsets           → List[Tuple[int, int]]  # 원문에서의 문자 위치 (start, end)
├── .attention_mask     → List[int]             # 마스크 (패딩=0, 토큰=1)
├── .type_ids           → List[int]             # segment ID (문장쌍 구분)
└── ...                 → 기타 속성들
```



In [14]:
print('첫 번째 샘플의 토큰화 결과 :', train_encodings[0].tokens)

첫 번째 샘플의 토큰화 결과 : ['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현', '##악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '##다', '

In [15]:
print('첫 번째 샘플의 길이 :', len(train_encodings[0].tokens))

첫 번째 샘플의 길이 : 512


In [16]:
print('첫 번째 샘플의 정수 인코딩 :', train_encodings[0].ids)

첫 번째 샘플의 정수 인코딩 : [2, 13934, 2236, 2440, 27982, 2259, 21310, 2079, 11994, 3791, 2069, 3790, 1508, 2088, 636, 3800, 2170, 3717, 2052, 9001, 8345, 4642, 2200, 3689, 3657, 2079, 19282, 2069, 1363, 2370, 2259, 936, 2069, 554, 2259, 2062, 18, 1504, 4342, 27982, 2259, 13934, 2196, 2440, 2170, 1195, 23260, 6233, 17370, 2113, 2165, 2069, 809, 1, 3706, 2052, 2181, 8642, 2145, 7334, 2170, 4983, 2371, 4007, 1065, 5917, 2386, 2559, 4443, 2138, 4026, 2259, 11994, 3791, 2079, 15864, 2170, 5487, 2371, 4683, 3605, 18, 3819, 5986, 27135, 1376, 2645, 2203, 2292, 2079, 5872, 2200, 5986, 4152, 2252, 22835, 16706, 2052, 5485, 2205, 2259, 17087, 2079, 19282, 29, 2517, 2069, 881, 2088, 652, 2073, 23404, 2069, 1122, 2886, 13964, 16, 3982, 2052, 9944, 21, 2429, 2170, 11994, 3791, 2079, 1258, 2465, 6233, 24294, 1504, 3967, 2170, 4027, 2052, 5121, 3979, 2069, 18274, 21575, 23548, 575, 2073, 5292, 2085, 7251, 2116, 1415, 2062, 18, 3776, 2079, 942, 2286, 2446, 4196, 2079, 3640, 6509, 636, 2079, 4450, 2170, 10329, 

In [17]:
print('첫 번째 샘플의 토큰화 결과 :', train_encodings[0].attention_mask)

첫 번째 샘플의 토큰화 결과 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [18]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []

    for i in tqdm(range(len(answers))):
        # 토큰화 전 문자에서 인덱스(start,end)가 토큰화 후 달라질 것이기 때문에 찾아내야한다
        # char_to_token은 주로 자연어처리(NLP)에서 사용되는 개념으로, 문자(character) 위치를 토큰(token) 위치로 변환하는 기능
        # char_to_token(i, char_pos)
        # i: 배치 내 문장 인덱스
        # char_pos: 원본 텍스트에서 문자의 인덱스 (정수)
        # 반환값: 해당 문자가 속한 토큰의 인덱스 (정수 또는 None)
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        # 파이썬 슬라이싱 규칙 - 1 해주기
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # 시작 인덱스가 비정상인 경우 -> 본문에 정답이 없는 경우
        if start_positions[-1] is None:
            # model_max_length -> 해당 토크나이저가 사용하는 모델의 최대 토큰 길이
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        # 종료 인덱스가 비정상인 경우 -> 본문에 정답이 없는 경우
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
              deleting_list.append(i)

    # update 메서드(dict의 메서드) -> 기존의 BatchEncoding에 새로운 키-값 쌍을 추가할 때 사용
    # BatchEncoding은 transformers에서 제공하는 객체이지만, 내부적으로 딕셔너리처럼 다룰 수 있도록 설계돼 있어 update()를 사용할 수 있습니다.
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [19]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test= add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 332198.17it/s]


In [20]:
print('삭제 예정인 훈련 샘플 :', deleting_list_for_train)
print('삭제 예정인 테스트 샘플 :', deleting_list_for_test)

삭제 예정인 훈련 샘플 : [711, 726, 728, 729, 761, 765, 767, 768, 805, 2586, 2587, 2722, 2724, 2725, 2731, 3392, 3475, 3478, 3491, 3495, 3498, 3919, 4462, 4465, 4513, 4515, 4565, 4765, 4766, 4772, 4774, 4779, 5334, 6603, 6638, 6639, 6748, 6749, 6750, 6765, 6766, 6771, 6776, 6897, 6898, 6900, 7739, 7741, 9203, 9211, 10880, 11039, 11212, 11727, 11776, 11788, 11789, 11791, 12168, 13708, 13711, 13996, 14460, 14461, 14491, 14724, 14729, 14885, 15764, 15970, 15971, 15973, 15974, 15976, 15977, 15979, 15980, 16080, 17683, 17815, 17828, 18389, 18392, 19045, 19052, 19053, 19195, 19636, 19637, 19638, 19640, 19656, 19761, 19764, 19765, 20614, 20618, 21224, 21243, 21334, 21335, 21338, 21361, 21521, 21522, 22627, 22633, 24003, 24577, 24579, 24580, 24768, 25108, 25176, 25182, 25185, 25186, 25187, 25188, 25448, 25451, 25454, 25457, 25460, 27105, 27112, 27113, 27114, 27159, 27293, 27295, 27555, 27558, 28025, 28438, 28779, 29162, 29189, 29289, 29290, 29855, 31889, 31890, 31891, 31894, 31905, 32050, 32051, 32057, 

In [21]:
# 여기서 정답이 있지만 삭제 예정 샘플에 찍히는 이유는 512에서 자르기 때문에 정답 라벨이 그 뒤에 있기 때문
print('761번 샘플의 질문 :', train_questions[761])
print('*'*80)
print('761번 샘플의 본문 :', train_contexts[761])
print('*'*80)
print('761번 샘플의 정답 :', train_answers[761])

761번 샘플의 질문 : 소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최종 성적은?
********************************************************************************
761번 샘플의 본문 : 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름(21)은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영(25-강원도청)이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영(24-전북도청)은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트(네덜란드)가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바(체코-4분01초95)와 3위 올가 그라프(러시아-4분03초47)에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트(1분54초09)에게 돌아갔고 동메달은 하를로터 판바이크(1분54초54)가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라(1분56초40)가 차지했다. 김보름은 1분59초78로 21

In [22]:
# 정답 라벨이 있는지 확인하기 위해서 정수 인코딩 한걸 다시 되돌려서 확인(761번 샘플)
print('761번 샘플 전처리 후 :', tokenizer.decode(train_encodings['input_ids'][761]))

761번 샘플 전처리 후 : [CLS] 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름 ( 21 ) 은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영 ( 25 - 강원도청 ) 이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영 ( 24 - 전북도청 ) 은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트 ( 네덜란드 ) 가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바 ( 체코 - 4분01초95 ) 와 3위 올가 그라프 ( 러시아 - 4분03초47 ) 에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트 ( 1분54초09 ) 에게 돌아갔고 동메달은 하를로터 판바이크 ( 1분54초54 ) 가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라 ( 1분56초40 ) 가 차지했다. 김보름은 1분59초78로 21위에 올랐다. 노선영 ( 25 - 강원도청 ) 은 2분01초07로 29위, 양신영 ( 24 - 전북도청 ) [SEP] 소치 팀추월 파이널D

In [23]:
# 이제 문제가 있는 샘플들은 제거
def delete_samples(encodings, deleting_list):
  # 정수인코딩된 데이터들을 numpy 배열로 바꾸고 -> deleting_list(삭제할 행의 인덱스를 답고 있는 리스트)에 있는 행들을 지운다
  # np.delete(arr, obj, axis=None)
  # arr	삭제할 대상이 되는 배열
  # obj	삭제할 인덱스 또는 인덱스 리스트
  # axis	삭제할 축: 0이면 행(row), 1이면 열(column) / 생략시 1차원으로 펼쳐진 상태에서 삭제
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  # 똑같이 나머지 값들도
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [24]:
# 데이터 정제 -> max_length 초과 샘플들 삭제
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings,deleting_list_for_test)

In [25]:
print('------------삭제 전------------')
print('훈련 데이터의 샘플의 개수 :', len(train_contexts))
print('테스트 데이터의 샘플의 개수 :', len(val_contexts))

print('------------삭제 후------------')
print('훈련 데이터의 샘플의 개수 :', len(X_train[0]))
print('테스트 데이터의 샘플의 개수 :', len(X_test[0]))


------------삭제 전------------
훈련 데이터의 샘플의 개수 : 60407
테스트 데이터의 샘플의 개수 : 5774
------------삭제 후------------
훈련 데이터의 샘플의 개수 : 60140
테스트 데이터의 샘플의 개수 : 5717


In [26]:
# 기존의 모델 정의 했던 모델 변형
from transformers import TFBertModel

class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        # 최종결과 768(Bert의 히든) -> 2(시작 점수, 종료 점수)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                               kernel_initializer=tf.keras.
                                               initializers.TruncatedNormal
                                               (0.02),
                                               name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)

        # BERT가 토큰마다 점수를 출력하고, squeeze로 불필요한 1차원 제거 후, softmax로 확률로 바꿔서 어느 토큰이 정답인지 판단한다.
        # BERT의 마지막 층의 모든 토큰들
        sequence_output = outputs[0]

        # 768 -> 2
        logits = self.qa_outputs(sequence_output)

        # 사용할 출력층은 총 2개, 각각 시작 인덱스 예측과 종료 인덱스 예측에 사용
        # axis=-1이므로 마지막 차원을 기준으로 두 개의 텐서로 나눔
        # 결과:
        # start_logits.shape == (batch_size, seq_len, 1)
        # end_logits.shape == (batch_size, seq_len, 1)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        # start_logits = (batch_size, sequence_length,)
        # end_logits = (batch_size, sequence_length,)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        # 시작 인덱스에 대한 다음 클래스 분류 문제
        start_probs = self.softmax(start_logits)

        # 종료 인덱스에 대한 다중 클래스 분류 문제
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [27]:
# 모델 생성 및 컴파일
model = TFBertForQuestionAnswering("klue/bert-base")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
# softmax를 통과해서 왔으니 from_logits=False
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# loss=[loss, loss]는 모델 출력이 2개일 때, 각 출력에 대응되는 loss를 지정해주는 것
model.compile(optimizer=optimizer, loss=[loss, loss])

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

In [ ]:
# 모델 학습
history = model.fit(
    X_train, y_train,
    epochs=3,
    verbose=1,
    batch_size=16,
)

Epoch 1/3


In [ ]:
def predict_test_data_by_idx(idx):
  # 예시) "나는 사과를 좋아해요. [SEP] 무엇을 좋아하나요?"
  # split -> ["나는 사과를 좋아해요.", "무엇을 좋아하나요?"]
  # 0 -> 본문
  # 1 -> 질문
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  # 파이썬 슬라이싱에 의해 뺏던 1다시 더하기
  # 정답 인코딩을 슬라이싱으로 가져온다
  answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
  print('정답 :',tokenizer.decode(answer_encoded))
  # tf.constant(...) -> 리스트/넘파이 배열 -> Tensorflow Tensor로 변환
  # 차원을 하나 늘려서 배치(batch) 차원을 추가하는 표현
  # 예시)
  # tf.constant([101, 1234, 3456, 102, 0, 0, 0]).shape
  # (7,) ← 단일 문장, 1D 텐서
  # tf.constant(...)[None, :].shape
  # (1, 7) ← 배치 크기 1인 2D 텐서로 바뀜
  # 왜 차원을 늘려야 할까?
  # TensorFlow 모델은 입력을 항상 (batch_size, sequence_length) 형태로 기대하기 때문
  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
  # squeeze는 텐서의 불필요한 차원(크기가 1인 차원)을 제거하는 함수
  # softmax를 통과하고 나온게 각각의 결과 -> start_probs = [0], end_probs = [1] -> shape = (batch_size, sequence_length,)
  # 여기서 squeeze를 통해 sequencelength 만 뽑아낸다.
  # 그 중 제일 값이 큰 것을 시작 토큰/ 종료 토큰으로 각각 받아온다
  # 여기서 end 에서의 +1은 파이썬 슬라이싱 때문이다
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  # 이 start랑 end 값이 정수 인코딩된 값 기준이기 때문에 -> 그 값 기준으로 해당하는 라벨 토큰을 가져오고 다시 decode하는 과정을 거친다.
  answer_encoded = X_test[0][idx][start:end]
  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [ ]:
for i in range(0, 100):
  predict_test_data_by_idx(i)